In [1]:
import os, subprocess
import json
import uproot
import awkward as ak
import numpy as np
from coffea import processor, util, hist
import pandas as pd
import pickle
import matplotlib.pyplot as plt

In [2]:
with open('lumi.json') as f:
  lumis = json.load(f)

with open('xsec.json') as f:
  xs = json.load(f)

with open('pmap.json') as f:
  pmap = json.load(f)

ddbthr = 0.64

In [3]:
year='2016APV'
picklename = year+'/templates.pkl'
if not os.path.isfile(picklename):
    print("You need to create the pickle "+picklename)
else:
    templates = pickle.load(open(picklename,'rb')).sum('msd1','pt1').integrate('systematic','nominal').integrate('mjj',overflow='allnan')

In [4]:
templates.identifiers('process')

[<StringBin (EWKW) instance at 0x7fa6f0599df0>,
 <StringBin (EWKZ) instance at 0x7fa6f0599130>,
 <StringBin (QCD) instance at 0x7fa6f053eaf0>,
 <StringBin (VBF) instance at 0x7fa671177040>,
 <StringBin (VBFDipoleRecoilOff) instance at 0x7fa6f05a47f0>,
 <StringBin (VV) instance at 0x7fa6f0599f10>,
 <StringBin (WH) instance at 0x7fa6711770a0>,
 <StringBin (Wjets) instance at 0x7fa6f053e700>,
 <StringBin (ZH) instance at 0x7fa671177100>,
 <StringBin (Zjets) instance at 0x7fa6f053e820>,
 <StringBin (ZjetsHT) instance at 0x7fa6f0599910>,
 <StringBin (data) instance at 0x7fa6711771c0>,
 <StringBin (ggF) instance at 0x7fa6f05a4f40>,
 <StringBin (muondata) instance at 0x7fa671177220>,
 <StringBin (singlet) instance at 0x7fa6f053e160>,
 <StringBin (ttH) instance at 0x7fa671177160>,
 <StringBin (ttbar) instance at 0x7fa6f053e520>]

In [5]:
mc = ['QCD','Wjets','Zjets','Zjetsbb','EWKW','EWKZ','ttbar','singlet','VV','ggF','VBF','WH','ZH','ttH']
cutflow = pd.DataFrame(columns=['ggffail','ggfpass','vbffail','vbfpass'],index=mc)

In [6]:
templates.integrate('process','QCD').integrate('region',['signal-ggf','signal-vbf']).sum('genflavor','ddb1').values()

{(): 1705270.9551230483}

In [7]:
for p in mc:
    print(p)
    if "Zjets" in p:
        continue
    
    cutflow['ggffail'][p] = templates.sum('genflavor').integrate('ddb1',int_range=slice(0,ddbthr)).integrate('region','signal-ggf').integrate('process',p).values()[()]
    cutflow['ggfpass'][p] = templates.sum('genflavor').integrate('ddb1',int_range=slice(ddbthr,1)).integrate('region','signal-ggf').integrate('process',p).values()[()]
    cutflow['vbffail'][p] = templates.sum('genflavor').integrate('ddb1',int_range=slice(0,ddbthr)).integrate('region','signal-vbf').integrate('process',p).values()[()]
    cutflow['vbfpass'][p] = templates.sum('genflavor').integrate('ddb1',int_range=slice(ddbthr,1)).integrate('region','signal-vbf').integrate('process',p).values()[()]
    
cutflow['ggffail']['Zjets'] = templates.integrate('genflavor',int_range=slice(0,3)).integrate('ddb1',int_range=slice(0,ddbthr)).integrate('region','signal-ggf').integrate('process','Zjets').values()[()]
cutflow['ggfpass']['Zjets'] = templates.integrate('genflavor',int_range=slice(0,3)).integrate('ddb1',int_range=slice(ddbthr,1)).integrate('region','signal-ggf').integrate('process','Zjets').values()[()]
cutflow['vbffail']['Zjets'] = templates.integrate('genflavor',int_range=slice(0,3)).integrate('ddb1',int_range=slice(0,ddbthr)).integrate('region','signal-vbf').integrate('process','Zjets').values()[()]
cutflow['vbfpass']['Zjets'] = templates.integrate('genflavor',int_range=slice(0,3)).integrate('ddb1',int_range=slice(ddbthr,1)).integrate('region','signal-vbf').integrate('process','Zjets').values()[()]

cutflow['ggffail']['Zjetsbb'] = templates.integrate('genflavor',int_range=slice(3,4)).integrate('ddb1',int_range=slice(0,ddbthr)).integrate('region','signal-ggf').integrate('process','Zjets').values()[()]
cutflow['ggfpass']['Zjetsbb'] = templates.integrate('genflavor',int_range=slice(3,4)).integrate('ddb1',int_range=slice(ddbthr,1)).integrate('region','signal-ggf').integrate('process','Zjets').values()[()]
cutflow['vbffail']['Zjetsbb'] = templates.integrate('genflavor',int_range=slice(3,4)).integrate('ddb1',int_range=slice(0,ddbthr)).integrate('region','signal-vbf').integrate('process','Zjets').values()[()]
cutflow['vbfpass']['Zjetsbb'] = templates.integrate('genflavor',int_range=slice(3,4)).integrate('ddb1',int_range=slice(ddbthr,1)).integrate('region','signal-vbf').integrate('process','Zjets').values()[()]



QCD
Wjets
Zjets
Zjetsbb
EWKW
EWKZ
ttbar
singlet
VV
ggF
VBF
WH
ZH
ttH


In [8]:
cutflow.astype('int')

,ggffail,ggfpass,vbffail,vbfpass
QCD,1626891,4370,73502,506
Wjets,41289,140,1023,3
Zjets,14444,75,359,2
Zjetsbb,1834,560,41,13
EWKW,1868,17,320,4
EWKZ,357,16,58,4
ttbar,5303,51,157,1
singlet,894,10,43,0
VV,587,11,16,0
ggF,33,11,1,0


In [9]:
(cutflow['ggffail']['ggF']+cutflow['ggfpass']['ggF']+cutflow['ggffail']['VBF']+cutflow['ggfpass']['VBF'])/sum(cutflow['ggffail']+cutflow['ggfpass'])

3.4566973746743646e-05

In [10]:
(cutflow['vbffail']['ggF']+cutflow['vbfpass']['ggF']+cutflow['vbffail']['VBF']+cutflow['vbfpass']['VBF'])/sum(cutflow['vbffail']+cutflow['vbfpass'])

0.00013822717466667418

In [11]:
cutflow.astype('int').to_latex(buf=year+'/cutflow-final.tex')

/tmp/ipykernel_29739/3630590848.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  cutflow.astype('int').to_latex(buf=year+'/cutflow-final.tex')
